# Análisis de los diferentes modelos

## 1. Importación de librerías y modulos

In [ ]:
#%pip install -r requirements.txt

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader, PyPDFLoader
from langchain_community.llms import Ollama
from langchain_chroma import Chroma
from itertools import islice
import tqdm
from module import *

In [ ]:
emb_model = [
    "multilingual_large",
    "baai_large",
    "mxbai_large",
    "baai_small"
]

llm_model = [
    "llama3",
    "mistral",
    "phi3",
    "gemma"
]

## 2. Carga de datos 

In [ ]:
documents_loader = PyPDFDirectoryLoader("./data/test2")
documents = documents_loader.load()
print(f"Loaded {len(documents)} documents")

## 3. Creación de la base de datos vectorial

In [ ]:
# Load the existing database.
db = Chroma(
    collection_name="test2",
    persist_directory="./database",
    embedding_function=get_embedding_function(emb_model[3])
)

In [ ]:
# Adding documents with a progress bar
with tqdm.tqdm(total=len(documents), desc="Adding documents") as pbar:
    for document in documents:
        db.add_documents([document])
        pbar.update(1)

## 4. Preguntas de interés

Formulamos las preguntas y buscamos en la base de datos los chunks que nos ofrezcan mejor contexto para responderlas.

In [ ]:
questions_docs_cat = [
    "Quines són les principals causes de l'augment del PIB de Catalunya el 2022 segons l'informe financer?",
    "Quins sectors econòmics han tingut una evolució més positiva i quins han patit més durant el 2022?",
    "Quina és la importància del turisme estranger per a l'economia catalana segons l'informe?"
    "Com es distribueixen les competències entre l'Estat i la Generalitat de Catalunya segons l'informe financer?",
    "Com afecta la capacitat fiscal a la distribució de recursos entre comunitats autònomes?",
    "Quin impacte han tingut les transferències estatals en els ingressos no financers de la Generalitat de Catalunya el 2022?"
]

questions_docs_en = [
    "What were Accenture's total revenues and operating margins for the fourth quarter of fiscal 2023?",
    "What were the new bookings for Accenture in the fourth quarter of fiscal 2023, and how did they compare to the previous year?",
    "What revenue trends were observed in Accenture's key geographic markets during the fourth quarter of fiscal 2023?",
    "What changes occurred in NTT Data's total assets and equity between March 31, 2024, and June 30, 2024?",
    "How did NTT Data's consolidated financial results for the fiscal year ending March 31, 2025, compare to the previous fiscal year?",
    "What were the trends in NTT Data's cash flows from operating, investing, and financing activities during the three months ended June 30, 2024?"
]

questions_docs_es = [
    "¿Cuáles son los principales retos estructurales que enfrenta la economía española según el informe del Banco de España?",
    "¿Cómo se ha comportado la inflación en España durante 2023 y cuáles son las perspectivas para los próximos años?",
    "¿Cómo se describe el proceso de envejecimiento poblacional en España y cuáles son sus implicaciones económicas?",
    "¿Qué impacto ha tenido la volatilidad de los precios del petróleo en los resultados financieros de Repsol?",
    "¿Cómo ha afectado la transición energética a las estrategias y operaciones de Repsol?",
    "¿Qué medidas ha implementado Repsol para mejorar su sostenibilidad financiera y operativa?"
]

questions = questions_docs_cat + questions_docs_en + questions_docs_es

In [ ]:
contexts = []

for question in questions:
    # Get the top 5 most relevant documents
    results = db.similarity_search_with_score(question, k=5)

    # Make a list of the contexts
    question_contexts = []
    for doc, _score in results:
        question_contexts.append(doc.page_content)

    # Append the context sub-list to the list of contexts
    contexts.append(question_contexts)

print(contexts)

Ahora creamos las respuestas esperadas para cada pregunta.

In [ ]:
answers_cat = [
    "L'augment del PIB de Catalunya el 2022, que ha estat del 5,5%, s'atribueix a factors com la recuperació del consum de les llars, l'augment de la inversió en béns d'equipament i construcció, així com un fort increment de les exportacions totals, especialment impulsades pel turisme estranger.",
    "Els sectors que han tingut una evolució més positiva són els serveis, especialment el comerç, transport i hostaleria, amb un creixement del 7,9%. En canvi, la indústria ha patit una contracció del 2,0%, i el sector agrari ha experimentat una caiguda del 13,9%.",
    "El turisme estranger ha estat crucial per a l'economia catalana el 2022, amb un total de 14,9 milions de turistes i una despesa de 16.479 milions d'euros, representant el 20,7% del total de turistes estrangers a Espanya.",
    "Les competències es distribueixen en competències exclusives, compartides i executives, amb la Generalitat exercint un alt nivell d'autonomia en àrees com l'agricultura, cultura, dret civil, i ordenació territorial, entre altres.",
    "La capacitat fiscal determina que les comunitats autònomes amb menor capacitat rebin transferències positives per equilibrar els recursos necessaris per finançar els serveis públics fonamentals, mentre que aquelles amb major capacitat han de fer aportacions a les altres comunitats.",
    "Les transferències estatals van tenir un gran impacte el 2021 a causa de la despesa Covid, però el 2022, la seva reducció ha fet que els ingressos no financers tornin a dependre més del model de finançament autonòmic tradicional."
]

answers_en = [
    "Accenture's total revenues for the fourth quarter of fiscal 2023 were $16.0 billion, with a GAAP operating margin of 12.0%. The adjusted operating margin was 14.9%.",
    "New bookings for Accenture in the fourth quarter of fiscal 2023 were $16.6 billion, a decrease of 10% compared to the previous year.",
    "During the fourth quarter of fiscal 2023, revenues in North America remained flat in U.S. dollars but increased by 1% in local currency, while Europe saw a 10% increase in U.S. dollars and 7% in local currency, and Growth Markets experienced a 1% increase in U.S. dollars and 6% in local currency.",
    "Between March 31, 2024, and June 30, 2024, NTT Data's total assets increased from ¥29,604,223 million to ¥29,989,405 million, while total equity rose from ¥10,893,059 million to ¥11,068,578 million.",
    "NTT Data's consolidated financial results forecast for the fiscal year ending March 31, 2025, includes expected operating revenues of ¥13,460,000 million, a 0.6% increase, and a profit attributable to NTT of ¥1,100,000 million, a 14.0% decrease compared to the previous fiscal year.",
    "During the three months ended June 30, 2024, NTT Data's cash flows from operating activities decreased by ¥145,742 million compared to the previous year, while cash used in investing activities increased by ¥135,046 million, and cash provided by financing activities increased by ¥193,981 million."
]

answers_es = [
    "Los principales retos estructurales que enfrenta la economía española incluyen impulsar el crecimiento de la productividad, reducir la elevada tasa de paro estructural, incrementar el tamaño empresarial, mejorar la calidad y confianza en las instituciones, reforzar la sostenibilidad de las cuentas públicas, y afrontar los desafíos del envejecimiento poblacional y la transición ecológica.",
    "Durante 2023, la inflación en España continuó una moderación más intensa de lo esperado, impulsada principalmente por una evolución favorable de los precios de la energía. Se prevé que la inflación siga desacelerándose gradualmente en los próximos trimestres.",
    "El envejecimiento poblacional en España se caracteriza por una baja tasa de fecundidad y una alta esperanza de vida, lo que supone un desafío para el mercado laboral y las finanzas públicas debido al incremento del gasto en pensiones y sanidad. Se requieren políticas para promover la prolongación de la vida laboral y mejorar la formación continua."
    "La volatilidad de los precios del petróleo ha afectado los ingresos de Repsol, aunque la compañía ha logrado mitigar los impactos a través de estrategias de cobertura y diversificación de sus fuentes de ingresos.",
    "La transición energética ha llevado a Repsol a invertir significativamente en energías renovables y a reorientar sus operaciones hacia fuentes de energía más sostenibles, lo que refleja su compromiso con la reducción de emisiones de carbono.",
    "Repsol ha implementado medidas como la optimización de su portafolio de activos, la reducción de deuda y la inversión en tecnología para mejorar su sostenibilidad financiera y operativa a largo plazo."
]


answers = answers_cat + answers_en + answers_es
answers

Finalmente creamos el DataFrame para evaluar los modelos.

In [ ]:
data_samples = {
    'question': questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

## 5. Evaluación de los modelos

In [ ]:
embeddings = get_embedding_function(emb_model[3])
llm = Ollama(model=llm_model[0])

In [ ]:
import nest_asyncio
from ragas.run_config import RunConfig


nest_asyncio.apply()

try:
    score = evaluate(
        dataset,
        metrics=[faithfulness, answer_relevancy],
        llm=llm,
        embeddings=embeddings,
        raise_exceptions=False,
        run_config=RunConfig(
            #max_retries=30, # Default is 10
            #max_wait=180, # Default is 60
            #max_workers=64 # Default is 16
        )
    )
except Exception as e:
    print(f"An error ocurred: {e}")

df_score = score.to_pandas()
print(df_score)

In [ ]:
df_score.head(10)